# Защита персональных данных клиентов

**Задача:**<br>
Разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При этом после преобразования данных качество моделей машинного обучения не должно ухудшиться.

## Загрузка данных

In [1]:
#Импортируем библиотеки и модули
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [2]:
#Создадим датасет
df = pd.read_csv('insurance.csv')

In [3]:
#Выводим опервые 5 строк датасета
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
#Выводим общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков не обнаружено. Проверим данные на явные дубликаты.

In [5]:
#Проверим на явные дубликаты
print(df.duplicated().sum())

153


Обнаружены явные дубликаты. Удалим их.

In [6]:
#Удалим дубликаты
df.drop_duplicates(inplace=True)

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении признаков на обратимую матрицу качество линейной регрессии не изменится

**Обоснование:** Для обоснования найдём отношение предсказаний модели, сделанных по исходным признакам и по признакам, умноженным на обратимую матрицу

**Вариант 1**

В формуле обучения умножим матрицу X на обратимую матрицу P:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Раскроем скобки:

$$
w' = (P^T X^T XP)^{-1} P^T X^T y
$$

По свойству ассоциативности умножения матриц, получим:

$$
w' = P^{-1}(X^T X)^{-1}(P^T)^{-1} P^T X^T y
$$

Преобразуем произведение матриц в единичную матрицу:

$$
(P^T)^{-1} P^T = E
$$

Заменим произведение матриц в выражении на единичную матрицу:

$$
w' = P^{-1}(X^T X)^{-1}E X^T y
$$

$$
E X^T = X^T
$$

$$
w' = P^{-1}(X^T X)^{-1}X^T y
$$

По формуле исходной формуле обучения:

$$
(X^T X)^{-1} X^T y = w
$$

Тогда

$$
w' = P^{-1}w
$$

Обозначим a' как предсказания для матрицы признаков, домноженной на обратимую матрицу P. Тогда:

$$
a' = XPP^{-1}w
$$

Преобразуем произведение матриц в единичную матрицу:

$$
PP^{-1} = E
$$

Получим:

$$
a' = XEw = Xw = a
$$


**Варинт 2**

В формуле
$$
\arg\min_w MSE(XPw_p, y)
$$

Делаем замену w_p на преобразованную ранее формулу обучения:

$$
w_P = P^{-1}w
$$

Поулчаем:

$$
\arg\min_w MSE(XPP^{-1}w, y) = \arg\min_w MSE(XEw, y) = \arg\min_w MSE(Xw, y)
$$  

## Алгоритм преобразования

**Алгоритм**

1. Преобразовать признаки в матрицу F
2. Создать квадратную обратимую матрицу P из случайных чисел размеро n x n, где n - ширина матрицы признаков F
3. Умножить матрицу признаков F на обратимую матрицу P
4. Преобразовать целевой признак в вектор t

**Обоснование**

При умножении матрицы признаков на обратимую квадратную матрицу значения предсказаний модели "Линейная регрессия" не изменяться:

$$
a = a' = Xw = XPw
$$
Обозначения:

- $a$ — предсказания модели по исходным целевым признакам

- $a'$ — предсказания модели по целевым признакам, умноженным на обратимую квадратную матрицу

- $X$ — матрица ыелевых признаков

- $P$ — Обратимая квадратная матрица

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

## Проверка алгоритма

Найдём значение метрики R2 на непреобразованных признаках

In [7]:
#Выделим признаки и целевой признак
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

In [8]:
#Создадим модель линейной регрессии и получим среднее значение метрики R2 с помощью кросс-валидации
model = LinearRegression()
model.fit(features, target)
scores = cross_val_score(model, features, target, cv=5)
final_r2 = scores.mean()

print('Среднее значение метрики R2:', final_r2)

Среднее значение метрики R2: 0.42779425802804677


Преобразуем признаки в матрицу F, целевой признак в вектор t и найдём сред. значение R2 с помощью кросс-валидации

In [9]:
#Для проверки алгоритма преобразуем признаки в матрицу, а ц.п. в вектор
F = np.array(features)
t = np.array(target)

In [10]:
#Выведем признаки, преобразованные в матрицу, для визуальной проверки
print(F)

[[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 ...
 [0.00e+00 2.00e+01 3.39e+04 2.00e+00]
 [1.00e+00 2.20e+01 3.27e+04 3.00e+00]
 [1.00e+00 2.80e+01 4.06e+04 1.00e+00]]


In [11]:
#Создадим обратимую квадратную матрицу P и выведем произведение матриц F и P для визуальной проверки
P = np.random.randint(100, size=(F.shape[1], F.shape[1]))
print(F@P)

[[4464484. 3972064.  398537. 4515138.]
 [3420472. 3044543.  305913. 3459659.]
 [1890290. 1682842.  169160. 1911986.]
 ...
 [3051224. 2714030.  272146. 3085770.]
 [2943318. 2618272.  262723. 2976782.]
 [3654354. 3250790.  326017. 3695696.]]


Данные в матрице признаков F и в произведении матриц F и P сильно различаются. Значит, умножение признаков на обратимую квадратную матрицу можно использовать как защиту данных.

In [12]:
def get_rand_matrix():
    det = 0
    # провекра матрицы на обратимость
    while det == 0: 
        matrix = np.random.normal(size=(F.shape[1], F.shape[1]))
        det = np.linalg.det(matrix)
    return matrix

In [13]:
%%time

#Найдём среднее значение R2 для матрицы признаков F, умноженной на обратимую матрицу P 
#с помощью кросс-валидации

P = get_rand_matrix()
scores = cross_val_score(model, F@P, t, cv=5) #Получеам список из R2, полученных кросс-валидацией
final_r2_after = scores.mean()

print('Среднее значение метрики R2:', final_r2_after)

Среднее значение метрики R2: 0.4277942580281243
CPU times: total: 31.2 ms
Wall time: 33 ms


In [14]:
#Вычислим разницу в процентах метрик R2 до и после применения алгоритма преобразования
print((final_r2-final_r2_after)/final_r2_after*100)

-1.8127657586504795e-11


**Вывод**: Алгоритм преобразования не оказывает влияния на качество модели, т.к. различие в метрике R2 до и после применения алгоритма очень мало.

## Общий вывод

По условиям задачи необходимо было разработать алгоритм преобразования данных, чтобы по ним было сложно востановить персональную информацию о клиентах.<br><br>
По формулам обучения модели "Линейная регрессия" было установлено, что при умножении признаков (данных о клиентах) на обратимую квадратную матрицу, значения предсказаний модели не изменяться. Также был оустановлено, что по произведению матрциы признаков и обратимой квадратнйо матрицы сложно восстановить исходную информацию.<br><br>
На основе этих выводов был разработан алгоритм преобразования данных:<br><br>
**Алгоритм**

1. Преобразовать признаки в матрицу F
2. Создать квадратную обратимую матрицу P из случайных чисел размеро n x n, где n - ширина матрицы признаков F
3. Умножить матрицу признаков F на обратимую матрицу P
4. Преобразовать целевой признаки в вектор t